# Model and Conclusion

# Setup

In [1]:
# dependencies
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 300)
from IPython.display import display

from matplotlib import pyplot as plt
%matplotlib inline 

import seaborn as sns

import missingno as msno

from functools import reduce

In [2]:
# we explored turicrate but will not use it further
# import turicreate as tc

# Get data

In [3]:
# Panda - stored in memory (limited)
# df = pd.read_csv('../data/clean_table.csv')

In [4]:
# SFrame - better when you have to scale up
# base_table = tc.SFrame.read_csv('../data/base_table_time_slots.csv')
# activity_table = tc.SFrame.read_csv('../data/activity_table.csv')
base_table = pd.read_csv('../data/base_table_time_slots.csv')
activity_table = pd.read_csv('../data/activity_table.csv')
time_slot_table = pd.read_csv('../data/time_slot_table.csv')

# Data Exploration
## Standard analysis

In [5]:
base_table.shape

(3386, 8)

In [6]:
activity_table.shape

(3386, 92)

In [7]:
time_slot_table.shape

(90, 337)

In [8]:
# panda only
# df.dtypes

In [9]:
base_table.head(5)

,Unnamed: 0,id,activity_name,category_name,start_date,start_date_timestamp,end_date,end_date_timestamp
0,0,0,Trello,Personal Adjusting,Mon Oct 22 00:01:13 GMT+02:00 2018,1540159273005,Mon Oct 22 00:11:09 GMT+02:00 2018,1540159869559
1,1,1,Series / Docu,Entertainment,Mon Oct 22 00:11:09 GMT+02:00 2018,1540159869559,Mon Oct 22 01:00:20 GMT+02:00 2018,1540162820068
2,2,2,Sleep,Refresh,Mon Oct 22 01:00:20 GMT+02:00 2018,1540162820068,Mon Oct 22 08:24:18 GMT+02:00 2018,1540189458018
3,3,3,Moving - youtube,Transport,Mon Oct 22 08:24:18 GMT+02:00 2018,1540189458018,Mon Oct 22 08:32:29 GMT+02:00 2018,1540189949037
4,4,4,Trello,Personal Adjusting,Mon Oct 22 08:32:29 GMT+02:00 2018,1540189949037,Mon Oct 22 08:40:44 GMT+02:00 2018,1540190444165


In [10]:
activity_table.head(5)

,Unnamed: 0,Analyzing,Beatbox,Better Now,Bike,Celebration - Bar,Celebration - Club,Chess,Clean,Cut hair,Dance,Drink - Shit,Enjoying the moment,Food,Food - Order,Food - Prepare,Food - restaurant,Food - shit,Health - bad,Helpful Future,House cleaning,Karaoke,Learning - Article,Learning - Language,Learning - university,Life Overwhelmed,Massage,Movies,Moving - Analyzing,Moving - Social Media,Moving - learn,Moving - social,Moving - youtube,Museum,Org,Org - Money,Org - Paper,Piano,Playing,Purpose,Purpose - Creation,Purpose - Learning,Purpose - Teach,Purpose - Transpose Knowledge,Samples - Better humans,Series / Docu,Shopping real/online,Sleep,Social - Awesome,Social - Close,Social - Family,Social - Media,Social - New,Social - Real Life,Social - care of others,Social - girl,Special Place,Sport,Sunbath,Swimming,Table Tennis,Trello,Trip to City Center,Trip to Nature,University - Orga,University - Purpose,University - Task,University - lectures boring,University - overwhelmed,WC,WC - Play,WC - analyzing,WC - learn,Walking,Wasted,Wasted - Moving,Wasted - reading,Wasted - sleep,Wasted - social,Wasted - watching,Work - Learning,Work - Meeting,Work - Org,Work - Purpose,Work - Social,Work - Task planning,Work - wasted,Youtube,moving - groceries,moving - eat,untracked,activity_name
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Trello
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Series / Docu
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Sleep
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Moving - youtube
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Trello


In [11]:
time_slot_table.head()

,activity_name,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21,t22,t23,t24,t25,t26,t27,t28,t29,t30,t31,t32,t33,t34,t35,t36,t37,t38,t39,t40,t41,t42,t43,t44,t45,t46,t47,t48,t49,t50,t51,t52,t53,t54,t55,t56,t57,t58,t59,t60,t61,t62,t63,t64,t65,t66,t67,t68,t69,t70,t71,t72,t73,t74,t75,t76,t77,t78,t79,t80,t81,t82,t83,t84,t85,t86,t87,t88,t89,t90,t91,t92,t93,t94,t95,t96,t97,t98,t99,t100,t101,t102,t103,t104,t105,t106,t107,t108,t109,t110,t111,t112,t113,t114,t115,t116,t117,t118,t119,t120,t121,t122,t123,t124,t125,t126,t127,t128,t129,t130,t131,t132,t133,t134,t135,t136,t137,t138,t139,t140,t141,t142,t143,t144,t145,t146,t147,t148,...,t186,t187,t188,t189,t190,t191,t192,t193,t194,t195,t196,t197,t198,t199,t200,t201,t202,t203,t204,t205,t206,t207,t208,t209,t210,t211,t212,t213,t214,t215,t216,t217,t218,t219,t220,t221,t222,t223,t224,t225,t226,t227,t228,t229,t230,t231,t232,t233,t234,t235,t236,t237,t238,t239,t240,t241,t242,t243,t244,t245,t246,t247,t248,t249,t250,t251,t252,t253,t254,t255,t256,t257,t258,t259,t260,t261,t262,t263,t264,t265,t266,t267,t268,t269,t270,t271,t272,t273,t274,t275,t276,t277,t278,t279,t280,t281,t282,t283,t284,t285,t286,t287,t288,t289,t290,t291,t292,t293,t294,t295,t296,t297,t298,t299,t300,t301,t302,t303,t304,t305,t306,t307,t308,t309,t310,t311,t312,t313,t314,t315,t316,t317,t318,t319,t320,t321,t322,t323,t324,t325,t326,t327,t328,t329,t330,t331,t332,t333,t334,t335
0,Trello,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,2,2,3,3,1,0,0,1,0,1,1,1,1,1,1,1,1,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,2,2,1,1,0,0,0,1,2,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,1,2,3,0,1,1,0,0,0,0,1,0,0,0,0,0,1,2,0,1,1,0,1,0,0,...,0,0,0,1,1,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,1,1,2,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,2,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,1,0,0,0,0,0,0,1,0,0,1,1,1,1,2,2,4,3,4,2,1,2,3,2,0
1,Series / Docu,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,2,2,1,0,0,0,0,0,0,0,0,0,0,2,3,4,3,2,0,0,0,2,2,5,5,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,1,1,0,1,2,3,5,3,1,1,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,3,2,1,1,1,0,0,0,0,0,1,1,1,0,0,0,0,1,2,4,4,4,2,4,4,3,2,1,0,0,0,0,...,2,3,5,3,2,4,5,4,3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,1,1,2,3,2,1,0,1,1,1,1,1,0,0,0,0,1,1,3,3,2,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,3,2,3,2,1,1,0,1,1,1,1,1,1,1,2,2,3,2,1,1,2,2,2,1,2,1,4,4,3,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,1,1,2,2,2,3,3,4,5,4,4,2,1
2,Sleep,6,9,12,13,14,15,15,15,15,15,15,15,15,15,14,12,9,9,5,5,4,3,3,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,4,7,8,9,13,16,17,17,17,18,18,18,18,18,18,18,17,13,14,7,6,4,3,2,2,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,4,7,8,13,16,19,19,19,19,19,19,19,19,19,19,19,16,15,13,7,6,2,0,0,0,0,0,1,1,2,2,0,0,0,0,0,0,0,0,0,0,1,1,1,1,2,2,2,2,5,9,11,13,16,17,18,...,0,0,0,1,6,7,12,12,14,15,16,17,17,16,16,16,16,16,17,16,13,11,8,8,6,7,4,3,3,3,2,2,1,1,1,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,1,3,5,7,10,13,14,15,15,16,16,17,17,17,17,17,17,16,17,15,13,11,9,7,4,3,2,3,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,4,4,6,9,12,13,14,14,14,14,14,15,15,14,14,15,15,13,13,14,14,13,11,8,6,3,2,2,3,2,2,1,1,1,1,2,2,1,0,0,0,0,0,0,0,0,0,0,0
3,Moving - youtube,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,2,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Clean,0,2,1,0,0,0,0,0,0,0,0,0,0

# Is the data good enough for the model?

Conclusion: A recommender system needs a table with an ID, an activity ID and optionally ratings. 

- We have a table with IDs
- We have a table with Activity IDs 
- We have no ratings. 




We have a timeslot table where we can recommend an activity based on a timeslot.

# Split dataset

In [12]:
# for current model not necessary
# train, test = tc.recommender.util.random_split_by_user(df, 'category', 'activity') 

# Build model pipeline
## Time Slot Model
Count activty occurences for each time slot

In [13]:
# model = tc.recommender.create(base_table ,'id','activity_name', ranking=False)

In [14]:
class TimeSlotModel:
    
    def __init__(self, time_slots):
        self.mapping = {}        
        self.initialize_mapping()
        self.minute_interval = 30       
        self.amount_time_slots = (7 * 24 * 60) // self.minute_interval
        self.time_slots = time_slots
        
    def get_top_activities_by_timeslot(self, time_slot, amount = 3):
        most_occured_activities = self.time_slots.sort_values(by=time_slot,ascending=False)
        return most_occured_activities.head(amount)[['activity_name',time_slot]]
            
    def get_top_activities_of_week(self, amount = 3):
        top_most_occured_activities = [model_time.get_top_activities_by_timeslot('t{0}'.format(x))['activity_name'] 
                                       for x in range(self.amount_time_slots)]
        return top_most_occured_activities
    
    def display_top_activities_of_week(self, amount=3):
        top_most_occured_activities = self.get_top_activities_of_week()
        most_favorites = pd.DataFrame()
        day_n = 1
        for x in range(self.amount_time_slots):
            most_favorites['t{0}'.format(x)] = top_most_occured_activities[x].values
            if (x+1) % 48 == 0:
                print('day {0} of 7'.format(day_n))
                day_n += 1
                display(most_favorites)
                most_favorites = pd.DataFrame() 
    
    def get_time_slot_by_date(self,date):
        day = date[0:3]
        hour = int(date[11:13])
        minute = int(date[14:16])
        return self.mapping[day][hour][minute]
    
    def initialize_mapping(self):
        timeslot = 0
        days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        hours = [h for h in range(0,24)]
        minutes_slot = [[x for x in range(0,30)],
                             [x for x in range(30,60)]]
        
        for day in days:
            self.mapping[day] = {}
            for hour in hours:
                self.mapping[day][hour] = {}
                for minute_slot in minutes_slot:
                    for minute in minute_slot:
                        self.mapping[day][hour][minute] = timeslot
                    timeslot += 1

In [15]:
model_time = TimeSlotModel(time_slot_table)
model_time.mapping
date = 'Mon Oct 22 12:00:00 GMT+02:00 2018'
time_slot = model_time.get_time_slot_by_date(date)
model_time.get_top_activities_by_timeslot('t{0}'.format(time_slot))
model_time.amount_time_slots

336

In [16]:
timeslot = 't{0}'.format(300)
model_time.get_top_activities_by_timeslot(timeslot)

,activity_name,t300
2,Sleep,14
27,Social - Close,2
62,Celebration - Bar,1


In [17]:
model_time.display_top_activities_of_week()

day 1 of 7


,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21,t22,t23,t24,t25,t26,t27,t28,t29,t30,t31,t32,t33,t34,t35,t36,t37,t38,t39,t40,t41,t42,t43,t44,t45,t46,t47
0,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Clean,Wasted - Moving,Work - Meeting,Work - Meeting,Work - Meeting,Work - Meeting,Work - Meeting,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Moving - learn,Wasted - Moving,Food,Food,Food,Food,Helpful Future,Youtube,Youtube,Sleep,Sleep,Sleep
1,Series / Docu,Analyzing,Series / Docu,Trello,Trello,Trello,Trip to Nature,Trip to Nature,Trip to Nature,Trip to Nature,Wasted - watching,Wasted - watching,Learning - Language,Social - girl,Social - girl,Sport,Clean,Clean,Food,Sleep,Work - Social,Sleep,Work - Purpose,Work - Purpose,Trello,Food,WC,Work - Learning,Work - Learning,Work - Learning,Bike,Work - Learning,University - Purpose,Social - Real Life,Work - Learning,Work - Social,University - Purpose,Food,Series / Docu,Series / Docu,Purpose - Creation,Social - Real Life,Social - Real Life,Helpful Future,Social - girl,University - Purpose,Learning - university,Series / Docu
2,Trello,Clean,Social - girl,Analyzing,Trip to Nature,Trip to Nature,Trello,Trello,Trello,Wasted - watching,Trello,Trello,Wasted - watching,Learning - Language,Food - Prepare,Learning - Language,Food,Food,Sleep,Work - Social,Sleep,Work - Purpose,Sleep,Learning - Language,Food,Trello,Work - Learning,Learning - Article,Food,Analyzing,Wasted,University - Purpose,Work - Social,University - Purpose,University - Purpose,University - Purpose,Moving - youtube,Bike,Social - girl,Social - Media,Series / Docu,Better Now,Wasted,Social - girl,House cleaning,Playing,University - Purpose,Wasted


day 2 of 7


,t48,t49,t50,t51,t52,t53,t54,t55,t56,t57,t58,t59,t60,t61,t62,t63,t64,t65,t66,t67,t68,t69,t70,t71,t72,t73,t74,t75,t76,t77,t78,t79,t80,t81,t82,t83,t84,t85,t86,t87,t88,t89,t90,t91,t92,t93,t94,t95
0,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Food,Clean,Clean,Work - Meeting,Work - Meeting,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Work - Social,Work - Social,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Work - Purpose,Helpful Future,Helpful Future,Purpose - Teach,Purpose - Teach,Purpose - Teach,Series / Docu,Purpose - Teach,Learning - university,Social - girl,Food,Sleep,Sleep,Sleep
1,Series / Docu,Clean,Clean,Moving - social,Moving - social,Moving - social,Social - Close,Celebration - Bar,Moving - social,Moving - social,Playing,Youtube,Learning - Article,Social - Close,Clean,Food - Prepare,Sleep,Sleep,Sleep,Work - Social,Work - Social,Work - Meeting,Work - Meeting,Work - Social,Work - Social,Food - Prepare,Work - Purpose,Work - Social,Purpose - Learning,Work - Meeting,Food,Learning - university,Learning - university,Work - Social,Learning - university,Work - Purpose,Wasted,Social - Real Life,Social - Real Life,Food,Purpose - Teach,Social - girl,Moving - social,Moving - social,Learning - university,Clean,Learning - university,Series / Docu
2,Clean,Series / Docu,Series / Docu,Series / Docu,Social - Close,Social - Close,Celebration - Bar,Trello,Celebration - Bar,Wasted - Moving,Trello,Playing,Health - bad,Clean,Learning - Language,Clean,Food - Prepare,Food,Food,Wasted - Moving,Work - Org,Work - Org,Trello,Work - Learning,Walking,Work - Purpose,Food,Food,Food,Trello,Purpose - Creation,Wasted,University - Purpose,Work - Meeting,Purpose - Creation,Wasted,Moving - Social Media,Wasted - Moving,Food,Series / Docu,University - Task,Series / Docu,Social - girl,Clean,Food - Prepare,Food,Clean,Social - girl


day 3 of 7


...

,t192,t193,t194,t195,t196,t197,t198,t199,t200,t201,t202,t203,t204,t205,t206,t207,t208,t209,t210,t211,t212,t213,t214,t215,t216,t217,t218,t219,t220,t221,t222,t223,t224,t225,t226,t227,t228,t229,t230,t231,t232,t233,t234,t235,t236,t237,t238,t239
0,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Food,Sleep,Food,Food,Purpose - Learning,University - Purpose,University - Purpose,University - Purpose,University - Purpose,Purpose - Learning,Purpose - Learning,Series / Docu,Learning - university,Purpose - Learning,Food,Food,Moving - social,Moving - social,Moving - social,Moving - social,Moving - social,Moving - social,Moving - social,Moving - social,Food,Food,Food
1,Series / Docu,Series / Docu,Series / Docu,Social - Family,Clean,Clean,Clean,Moving - social,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Clean,Clean,Food,Food,Wasted - watching,Wasted - Moving,Wasted - Moving,Sport,Sleep,Walking,Sleep,Purpose,Sleep,Trello,Purpose - Learning,Purpose - Learning,Walking,University - Purpose,University - Purpose,Purpose - Learning,Social - Media,Purpose - Creation,Purpose - Creation,Moving - social,Food,Sport,Food,Food,Social - Real Life,Food,moving - groceries,Food,Moving - social,Social - Close,Social - Real Life
2,Social - Close,Trello,Trello,Series / Docu,Social - Family,Analyzing,Social - Close,Social - Close,Social - Family,Social - Family,Social - Family,Social - Family,Social - Family,Social - Family,WC,Social - Close,Social - girl,Clean,Wasted - watching,Wasted - watching,Food,Wasted - watching,Food,Walking,Purpose - Learning,Social - Family,Purpose,Food,Clean,Purpose - Learning,WC,WC,Purpose - Transpose Knowledge,Moving - Analyzing,University - Purpose,Purpose - Learning,Food - shit,Social - Family,Social - girl,moving - groceries,Food - Prepare,Food - Prepare,Social - Real Life,Clean,moving - groceries,moving - groceries,Social - Real Life,Moving - social


day 6 of 7


,t240,t241,t242,t243,t244,t245,t246,t247,t248,t249,t250,t251,t252,t253,t254,t255,t256,t257,t258,t259,t260,t261,t262,t263,t264,t265,t266,t267,t268,t269,t270,t271,t272,t273,t274,t275,t276,t277,t278,t279,t280,t281,t282,t283,t284,t285,t286,t287
0,Moving - social,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Food,Food,Food,Clean,Purpose - Learning,Learning - university,Purpose - Learning,Learning - university,Learning - university,Walking,Moving - Social Media,moving - groceries,Moving - social,Food - shit,Food,Food,Learning - university,Learning - university,Moving - social,Celebration - Bar,Social - Real Life,Celebration - Bar,Social - Real Life,Sleep
1,Sleep,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Series / Docu,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Clean,Clean,Clean,Social - girl,Social - girl,Sunbath,Sleep,Social - girl,Sleep,Purpose - Learning,Clean,Purpose - Learning,Learning - university,Walking,Walking,Learning - university,Purpose - Creation,Moving - social,moving - groceries,Walking,Series / Docu,Moving - social,Food,Food,Celebration - Bar,Moving - social,Youtube,Youtube,Celebration - Bar,Social - Real Life
2,Series / Docu,Learning - university,Trello,Clean,Clean,Series / Docu,Series / Docu,Social - Close,Series / Docu,Clean,Trello,Trello,Trello,Trello,Trello,Trello,Trello,Sport,Social - Close,Moving - social,Social - girl,Clean,Series / Docu,Food,Series / Docu,Trello,Social - Close,Food - Prepare,Food,Moving - social,Food - Prepare,Social - girl,Sport,Playing,Helpful Future,Moving - Social Media,Food,Food,Food - shit,Purpose - Creation,Moving - social,Celebration - Bar,Learning - university,Learning - university,Celebration - Bar,Food,Youtube,Celebration - Bar


day 7 of 7


,t288,t289,t290,t291,t292,t293,t294,t295,t296,t297,t298,t299,t300,t301,t302,t303,t304,t305,t306,t307,t308,t309,t310,t311,t312,t313,t314,t315,t316,t317,t318,t319,t320,t321,t322,t323,t324,t325,t326,t327,t328,t329,t330,t331,t332,t333,t334,t335
0,Series / Docu,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Food,Food,Walking,Food,Food,Moving - social,Learning - university,Social - girl,Social - girl,Social - girl,Social - girl,Moving - social,Analyzing,Moving - social,Trello,Moving - social,Trello,Series / Docu,Series / Docu,Series / Docu,Series / Docu,Social - Real Life,Food - shit
1,Sleep,Series / Docu,Series / Docu,Clean,Social - Close,Series / Docu,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,Social - Close,WC - analyzing,Moving - social,Social - girl,Social - girl,Social - Family,Trello,Clean,Food,Clean,Clean,Food - Prepare,Walking,Sleep,Social - Family,Social - Family,Food,Moving - social,Learning - university,Food,Learning - university,Analyzing,Analyzing,Trello,Trello,Food - shit,Food,Food,Social - girl,Food,Food - shit,Trello,Analyzing,Social - Real Life
2,Celebration - Bar,Celebration - Bar,Food,Food,Clean,Social - Close,Social - girl,Social - girl,Moving - social,Clean,Clean,Social - Family,Celebration - Bar,Celebration - Bar,Moving - social,WC - learn,WC - analyzing,WC,Wasted - social,Learning - Language,Learning - Language,Trello,Clean,Moving - social,Moving - social,Moving - Analyzing,Sleep,Food,Moving - social,Sleep,Learning - university,Food,Samples - Better humans,Learning - university,Sleep,Sleep,Learning - university,Food - shit,Food - shit,Moving - social,Series / Docu,Series / Docu,Social - Real Life,Analyzing,Trello,Youtube,Trello,Analyzing


In [18]:
time_slots = model_time.get_top_activities_of_week(0)
most_favorites = []
day_n = 1
for x in range(model_time.amount_time_slots):
    most_favorites.append(time_slots[x].values[0])

display(most_favorites)

['Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Clean',
 'Wasted - Moving',
 'Work - Meeting',
 'Work - Meeting',
 'Work - Meeting',
 'Work - Meeting',
 'Work - Meeting',
 'Work - Purpose',
 'Work - Purpose',
 'Work - Purpose',
 'Work - Purpose',
 'Work - Purpose',
 'Work - Purpose',
 'Work - Purpose',
 'Work - Purpose',
 'Work - Purpose',
 'Work - Purpose',
 'Work - Purpose',
 'Moving - learn',
 'Wasted - Moving',
 'Food',
 'Food',
 'Food',
 'Food',
 'Helpful Future',
 'Youtube',
 'Youtube',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Sleep',
 'Food',
 'Clean',
 'Clean',
 'Work - Meeting',
 'Work - Meeting',
 'Work - Purpose',
 'Work - Purpose',
 'Work - Purpose',
 'Work - Purpose',
 'Work - Social',
 'Work - Social',

time, monday, tuesday, ...
0:00-0:30 | sleep 0 | bla 48 | ..



In [19]:
columns = ['Time Slot','Monday','Tueday','Wednesday','Thursday','Friday','Saturday','Sunday']
week_table = pd.DataFrame(columns=columns)

idx = 0
n_slots_per_day = int(len(time_slots) / 7)

start_h = 0
start_m = '00'
end_h = 0
end_m = '30'

for idx in range(n_slots_per_day):
    week_table.loc[idx] = [
        "{}:{} - {}:{}".format(start_h,start_m,end_h,end_m),
        time_slots[idx].values[0],
        time_slots[idx+n_slots_per_day].values[0],
        time_slots[idx+n_slots_per_day*2].values[0],
        time_slots[idx+n_slots_per_day*3].values[0],
        time_slots[idx+n_slots_per_day*4].values[0],
        time_slots[idx+n_slots_per_day*5].values[0],
        time_slots[idx+n_slots_per_day*6].values[0]
    ]
    if idx % 2 == 1:
        start_h += 1
        start_m = '00'
        end_m = '30'
    else:
        start_m = '30'
        end_h += 1
        end_m = '00'
    idx += 1
    
    
week_table




,Time Slot,Monday,Tueday,Wednesday,Thursday,Friday,Saturday,Sunday
0,0:00 - 0:30,Sleep,Sleep,Sleep,Sleep,Sleep,Moving - social,Series / Docu
1,0:30 - 1:00,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep
2,1:00 - 1:30,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep
3,1:30 - 2:00,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep
4,2:00 - 2:30,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep
5,2:30 - 3:00,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep
6,3:00 - 3:30,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep
7,3:30 - 4:00,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep
8,4:00 - 4:30,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep
9,4:30 - 5:00,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep,Sleep


In [20]:
week_table.values

array([['0:00 - 0:30', 'Sleep', 'Sleep', 'Sleep', 'Sleep', 'Sleep',
        'Moving - social', 'Series / Docu'],
       ['0:30 - 1:00', 'Sleep', 'Sleep', 'Sleep', 'Sleep', 'Sleep',
        'Sleep', 'Sleep'],
       ['1:00 - 1:30', 'Sleep', 'Sleep', 'Sleep', 'Sleep', 'Sleep',
        'Sleep', 'Sleep'],
       ['1:30 - 2:00', 'Sleep', 'Sleep', 'Sleep', 'Sleep', 'Sleep',
        'Sleep', 'Sleep'],
       ['2:00 - 2:30', 'Sleep', 'Sleep', 'Sleep', 'Sleep', 'Sleep',
        'Sleep', 'Sleep'],
       ['2:30 - 3:00', 'Sleep', 'Sleep', 'Sleep', 'Sleep', 'Sleep',
        'Sleep', 'Sleep'],
       ['3:00 - 3:30', 'Sleep', 'Sleep', 'Sleep', 'Sleep', 'Sleep',
        'Sleep', 'Sleep'],
       ['3:30 - 4:00', 'Sleep', 'Sleep', 'Sleep', 'Sleep', 'Sleep',
        'Sleep', 'Sleep'],
       ['4:00 - 4:30', 'Sleep', 'Sleep', 'Sleep', 'Sleep', 'Sleep',
        'Sleep', 'Sleep'],
       ['4:30 - 5:00', 'Sleep', 'Sleep', 'Sleep', 'Sleep', 'Sleep',
        'Sleep', 'Sleep'],
       ['5:00 - 5:30', 'Sleep', 

# Second Model - Markov Chain
If Activiy A happend what is the most probable following activity.   
For markov model a permutation table is necessary. All activities are ordered by time so we can count the follow ups and norm them.


In [30]:
activities = base_table['activity_name']
unique_activities = activities.unique()
n_unique_activities = len(unique_activities)
n_unique_activities

90

In [33]:
transition_matrix = {}

for idx in range(len(activities)-1):
    state = activities[idx]
    transition = activities[idx+1]
    
    if state not in transition_matrix:
        transition_matrix[state] = {}

    
    if transition in transition_matrix[state]:
        transition_matrix[state][transition] += 1
    else:
        transition_matrix[state][transition] = 1
        
# then norm
for state in transition_matrix:
    total_sum = sum(transition_matrix[state].values())
    for transition in transition_matrix[state]:
        transition_matrix[state][transition] /= total_sum
        
transition_matrix

{'Trello': {'Series / Docu': 0.0449438202247191,
  'Moving - youtube': 0.011235955056179775,
  'WC ': 0.033707865168539325,
  'Social - Media': 0.0449438202247191,
  'Work - Social': 0.033707865168539325,
  'Learning - university': 0.011235955056179775,
  'Better Now': 0.033707865168539325,
  'Org ': 0.011235955056179775,
  'Social - Real Life': 0.056179775280898875,
  'Work - Purpose': 0.02247191011235955,
  'Analyzing': 0.1348314606741573,
  'University - lectures boring': 0.02247191011235955,
  'Trello': 0.033707865168539325,
  'University - Purpose': 0.011235955056179775,
  'Helpful Future': 0.011235955056179775,
  'Youtube': 0.02247191011235955,
  'Food': 0.033707865168539325,
  'Social - Close': 0.02247191011235955,
  'Wasted': 0.011235955056179775,
  'Sleep': 0.0898876404494382,
  'Work - Learning': 0.02247191011235955,
  'Moving - Analyzing': 0.02247191011235955,
  'Clean': 0.033707865168539325,
  'Moving - social': 0.033707865168539325,
  'WC - learn': 0.011235955056179775,
  

In [38]:
import operator
max(transition_matrix['Trello'].items(), key=operator.itemgetter(1))

('Analyzing', 0.1348314606741573)

[('Clean', 0.19205298013245034),
 ('WC ', 0.1390728476821192),
 ('WC - learn', 0.09271523178807947)]

In [50]:
from collections import Counter

class MarkovChainModel:
    def __init__(self, activities):
        self.transition_matrix = {}
        
        self.initialize_transition_matrix(activities)
        
    def get_top_transitions(self, state, n_top=3):
        counter = Counter(transition_matrix[state])
        return counter.most_common(n_top)

    def get_transition_matrix(self):
        return self.transition_matrix
        
    def initialize_transition_matrix(self,activities):
        transition_matrix = {}

        for idx in range(len(activities)-1):
            state = activities[idx]
            transition = activities[idx+1]

            if state not in transition_matrix:
                transition_matrix[state] = {}


            if transition in transition_matrix[state]:
                transition_matrix[state][transition] += 1
            else:
                transition_matrix[state][transition] = 1

        # then norm
        for state in transition_matrix:
            total_sum = sum(transition_matrix[state].values())
            for transition in transition_matrix[state]:
                transition_matrix[state][transition] /= total_sum
                
        self.transition_matrix = transition_matrix


In [54]:
activities = base_table['activity_name']
x = MarkovChainModel(activities)
# print(x.get_transition_matrix())
x.get_top_transitions('Sleep')

[('Clean', 0.19205298013245034),
 ('WC ', 0.1390728476821192),
 ('WC - learn', 0.09271523178807947)]

# Declare hyperparameter

# Fit and tune with cross-validation

# Evaluate Model

In [ ]:
# results = model.recommend(exclude_known=False)
# results

In [ ]:
#results = model.recommend(users=[1],exclude_known=False, )
#results

In [ ]:
#n_rows = 10
#activities = ['Sleep']
#similar_items = model.get_similar_items(activities, k=n_rows)
#similar_items.print_rows(n_rows)

# Select winner model

# Save winning model

In [ ]:
#model.save("../model/time_slot_recommendation.model")

# Load winning model

In [ ]:
#model = tc.load_model("../model/time_slot_recommendation.model")

# Communicate Results (Conclusion)

TODO